# Wokflow  **ZLightGBM** para competencia 3

Este workflow hace la predicción final en 202109 con un corte de undersampling de 0.05 entrenando desde 201901 hasta 202107, descartando 201910 y 202006

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-12-03 23:19:37 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657982,35.2,1454660,77.7,1355011,72.4
Vcells,1221409,9.4,8388608,64.0,1975153,15.1


In [2]:
PARAM <- list()
PARAM$experimento <- "e010_03"
PARAM$semilla_primigenia <- 100343

In [3]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [4]:
Sys.time()
require("data.table")

# leo los dos datasets y los uno
dataset_02 <- fread("~/buckets/b1/datasets/competencia_02_crudo.csv.gz")
dataset_03 <- fread("~/buckets/b1/datasets/competencia_03_crudo.csv.gz")

# los apilo uno debajo del otro
dataset <- rbindlist(
  list(dataset_02, dataset_03),
  use.names = TRUE,  
  fill = TRUE        
)

rm(dataset_02, dataset_03)
gc()

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]

# ordeno
setorder(dsimple, numero_de_cliente, periodo0)

# calculo topes
periodo_ultimo    <- dsimple[, max(periodo0)]
periodo_anteultimo <- periodo_ultimo - 1

# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n = 1:2, fill = NA, type = "lead"), by = numero_de_cliente
]

dsimple[periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA"]

# calculo BAJA+1
dsimple[periodo0 < periodo_ultimo &
          (is.na(periodo1) | periodo0 + 1 < periodo1),
        clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[periodo0 < periodo_anteultimo & (periodo0 + 1 == periodo1) &
          (is.na(periodo2) | periodo0 + 2 < periodo2),
        clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder(dsimple, pos)
dataset[, clase_ternaria := dsimple$clase_ternaria]

rm(dsimple)
gc()
Sys.time()

[1] "2025-12-03 23:19:45 UTC"

Loading required package: data.table



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,758333,40.5,1454660,77.7,1454660,77.7
Vcells,555645391,4239.3,1481535975,11303.3,1296634162,9892.6


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,767203,41.0,1454660,77.7,1454660,77.7
Vcells,560570692,4276.9,1481535975,11303.3,1296634162,9892.6


[1] "2025-12-03 23:20:05 UTC"

In [5]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### Eliminacion de Features

In [6]:
# Variables rotas
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

### Data Quality

Para los meses identificados donde para una variable todos sus valores son ceros, se aplica interpolación utilizando el mes previo y el posterior

In [7]:
library(DBI)
library(duckdb)
library(data.table)

con <- dbConnect(duckdb(), dbdir=":memory:")

corregir_interpolar <- function(dataset, campo, meses) {
  # Por si dataset viene con basura previa:
  # borramos v1, v2, promedio si existieran
  cols_tmp <- intersect(c("v1", "v2", "promedio"), names(dataset))
  if (length(cols_tmp) > 0) {
    dataset[, (cols_tmp) := NULL]
  }

  # Subo el dataset a DuckDB
  dbWriteTable(con, "tmp", dataset, overwrite = TRUE)

  meses_str <- paste(meses, collapse = ", ")

  query <- paste0("
    WITH base AS (
        SELECT
            *,
            lag(", campo, ") OVER (
                PARTITION BY numero_de_cliente
                ORDER BY foto_mes
            ) AS v1,
            lead(", campo, ") OVER (
                PARTITION BY numero_de_cliente
                ORDER BY foto_mes
            ) AS v2
        FROM tmp
    ),
    calc AS (
        SELECT
            *,
            CASE
                WHEN v1 IS NOT NULL AND v2 IS NOT NULL THEN (v1 + v2) / 2.0
                WHEN v1 IS NOT NULL THEN v1
                WHEN v2 IS NOT NULL THEN v2
                ELSE NULL
            END AS promedio
        FROM base
    )
    SELECT
        * EXCLUDE(v1, v2, promedio)
        REPLACE(
            CASE
                WHEN foto_mes IN (", meses_str, ") THEN promedio
                ELSE ", campo, "
            END AS ", campo, "
        )
    FROM calc
  ")

  result <- dbGetQuery(con, query)
  return(as.data.table(result))
}

In [8]:
# Campos que voy a arreglar sin 201910 y 202006
atributos_y_meses <- list(
  list(campo = "ctarjeta_visa_debitos_automaticos", meses = c(201904)),
  list(campo = "mttarjeta_visa_debitos_automaticos", meses = c(201904)),
  list(campo = "mrentabilidad", meses = c(201905)),
  list(campo = "mrentabilidad_annual", meses = c(201905)),
  list(campo = "mcomisiones", meses = c(201905)),
  list(campo = "mactivos_margen", meses = c(201905)),
  list(campo = "mpasivos_margen", meses = c(201905)),
  list(campo = "ccomisiones_otras", meses = c(201905)),
  list(campo = "mcomisiones_otras", meses = c(201905)),
  list(campo = "ccajeros_propios_descuentos", meses = c(202002, 202009, 202010, 202102)),
  list(campo = "mcajeros_propios_descuentos", meses = c(202002, 202009, 202010, 202102)),
  list(campo = "ctarjeta_visa_descuentos", meses = c(202002, 202009, 202010, 202102)),
  list(campo = "mtarjeta_visa_descuentos", meses = c(202002, 202009, 202010, 202102)),
  list(campo = "ctarjeta_master_descuentos", meses = c(202002, 202009, 202010, 202102)),
  list(campo = "mtarjeta_master_descuentos", meses = c(202002, 202009, 202010, 202102)),
  list(campo = "ccajas_depositos", meses = c(202105))
)

In [9]:
for (item in atributos_y_meses) {
  campo <- item$campo
  meses <- item$meses
  
  cat("Corrigiendo", campo, "en meses", paste(meses, collapse = ", "), "...\n")
  dataset <- corregir_interpolar(dataset, campo, meses)
}

Corrigiendo ctarjeta_visa_debitos_automaticos en meses 201904 ...
Corrigiendo mttarjeta_visa_debitos_automaticos en meses 201904 ...
Corrigiendo mrentabilidad en meses 201905 ...
Corrigiendo mrentabilidad_annual en meses 201905 ...
Corrigiendo mcomisiones en meses 201905 ...
Corrigiendo mactivos_margen en meses 201905 ...
Corrigiendo mpasivos_margen en meses 201905 ...
Corrigiendo ccomisiones_otras en meses 201905 ...
Corrigiendo mcomisiones_otras en meses 201905 ...
Corrigiendo ccajeros_propios_descuentos en meses 202002, 202009, 202010, 202102 ...
Corrigiendo mcajeros_propios_descuentos en meses 202002, 202009, 202010, 202102 ...
Corrigiendo ctarjeta_visa_descuentos en meses 202002, 202009, 202010, 202102 ...
Corrigiendo mtarjeta_visa_descuentos en meses 202002, 202009, 202010, 202102 ...
Corrigiendo ctarjeta_master_descuentos en meses 202002, 202009, 202010, 202102 ...
Corrigiendo mtarjeta_master_descuentos en meses 202002, 202009, 202010, 202102 ...
Corrigiendo ccajas_depositos en 

### Data Drifting

Deflacto montos con IPC utilizando 202106 como base. Utilizo como fuente: https://datos.gob.ar/series/api/series/?ids=148.3_INIVELNAL_DICI_M_26&utm_source=chatgpt.com&representation_mode=value&start_date=2020-01-01&end_date=2021-09-01&chartType=column  

In [10]:
drift_inf <- function(dataset, campos_monetarios, ind) {
  
  setDT(dataset)
  setDT(ind)
  
  # Merge para agregar coef por foto_mes
  dataset[ind, coef := i.coef, on = "foto_mes"]
  
  # Deflactar: monto_real = monto_nominal * coef
  for (campo in campos_monetarios) {
    dataset[, (campo) := get(campo) * coef]
  }
  
  # Quitar coef columna auxiliar
  dataset[, coef := NULL]
  
  invisible(dataset) 
}

In [11]:
ind <- data.table(
  foto_mes = c(
    201901, 201902, 201903, 201904, 201905, 201906,
    201907, 201908, 201909, 201910, 201911, 201912,
    202001, 202002, 202003, 202004, 202005, 202006,
    202007, 202008, 202009, 202010, 202011, 202012,
    202101, 202102, 202103, 202104, 202105, 202106,
    202107, 202108, 202109
  ),
  
  coef = c(
    2.550498, 2.457942, 2.348029, 2.270849, 2.202487, 2.144187,
    2.098139, 2.018280, 1.906113, 1.845309, 1.770002, 1.706181,
    1.668564, 1.630094, 1.582720, 1.549057, 1.535367, 1.502003,
    1.473492, 1.434759, 1.395188, 1.344603, 1.303434, 1.253239,
    1.204453, 1.162891, 1.109505, 1.066020, 1.031724, 1.000000,
    0.970889, 0.947511, 0.915043
  )
)

In [12]:
# Campos de montos
campos_monetarios <- names(dataset)[
  startsWith(names(dataset), "m") |
  startsWith(names(dataset), "Visa_m") |
  startsWith(names(dataset), "Master_m") |
  startsWith(names(dataset), "vm_m")
]

# Aplico
drift_inf(dataset, campos_monetarios, ind)

### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.

In [13]:
library(DBI)
library(duckdb)

fe_intrames <- function(df_fe) {
  # Aseguramos data.table / data.frame
  df_fe <- as.data.table(df_fe)
  
  # Conexión DuckDB en memoria
  con <- dbConnect(duckdb(), dbdir = ":memory:")
  on.exit(dbDisconnect(con, shutdown = TRUE))
  
  # Registrar el data.frame como tabla en DuckDB
  duckdb_register(con, "competencia_02", df_fe)
  
  # Macros
  dbExecute(con, "
    CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);
  ")
  
  dbExecute(con, "
    CREATE OR REPLACE MACRO div(a, b) AS 
      CASE 
        WHEN ifnull(b, 0) = 0 THEN NULL 
        ELSE ifnull(a, 0) / ifnull(b, 1) 
      END;
  ")
  
  # Query principal 
  query <- "
    WITH sumas AS (
        SELECT
            *
          , suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) AS tc_consumo_total
          , suma_sin_null(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) AS tc_financiacionlimite_total
          , suma_sin_null(Master_msaldopesos, Visa_msaldopesos) AS tc_saldopesos_total
          , suma_sin_null(Master_msaldodolares, Visa_msaldodolares) AS tc_saldodolares_total
          , suma_sin_null(Master_mconsumospesos, Visa_mconsumospesos) AS tc_consumopesos_total
          , suma_sin_null(Master_mconsumosdolares, Visa_mconsumosdolares) AS tc_consumodolares_total
          , suma_sin_null(Master_mlimitecompra, Visa_mlimitecompra) AS tc_limitecompra_total
          , suma_sin_null(Master_madelantopesos, Visa_madelantopesos) AS tc_adelantopesos_total
          , suma_sin_null(Master_madelantodolares, Visa_madelantodolares) AS tc_adelantodolares_total
          , suma_sin_null(tc_adelantopesos_total, tc_adelantodolares_total) AS tc_adelanto_total
          , suma_sin_null(Master_mpagado, Visa_mpagado) AS tc_pagado_total
          , suma_sin_null(Master_mpagospesos, Visa_mpagospesos) AS tc_pagadopesos_total
          , suma_sin_null(Master_mpagosdolares, Visa_mpagosdolares) AS tc_pagadodolares_total
          , suma_sin_null(Master_msaldototal, Visa_msaldototal) AS tc_saldototal_total
          , suma_sin_null(Master_mconsumototal, Visa_mconsumototal) AS tc_consumototal_total
          , suma_sin_null(Master_cconsumos, Visa_cconsumos) AS tc_cconsumos_total
          , suma_sin_null(Master_delinquency, Visa_delinquency) AS tc_morosidad_total
          , suma_sin_null(mplazo_fijo_dolares, mplazo_fijo_pesos) AS m_plazofijo_total
          , suma_sin_null(minversion1_dolares, minversion1_pesos) AS m_inversion1_total
          , suma_sin_null(mpayroll, mpayroll2) AS m_payroll_total
          , suma_sin_null(cpayroll_trx, cpayroll2_trx) AS c_payroll_total
          , suma_sin_null(
                suma_sin_null(suma_sin_null(cseguro_vida, cseguro_auto), cseguro_vivienda),
                cseguro_accidentes_personales
            ) AS c_seguros_total
        FROM competencia_02
    )
    SELECT
        sumas.*
      , div(m_plazofijo_total, cplazo_fijo) AS m_promedio_plazofijo_total
      , div(m_inversion1_total, cinversion1) AS m_promedio_inversion_total
      , div(mcaja_ahorro, ccaja_ahorro) AS m_promedio_caja_ahorro
      , div(mtarjeta_visa_consumo, ctarjeta_visa_transacciones) AS m_promedio_tarjeta_visa_consumo_por_transaccion
      , div(mtarjeta_master_consumo, ctarjeta_master_transacciones) AS m_promedio_tarjeta_master_consumo_por_transaccion
      , div(mprestamos_prendarios, cprestamos_prendarios) AS m_promedio_prestamos_prendarios
      , div(mprestamos_hipotecarios, cprestamos_hipotecarios) AS m_promedio_prestamos_hipotecarios
      , div(minversion2, cinversion2) AS m_promedio_inversion2
      , div(mpagodeservicios, cpagodeservicios) AS m_promedio_pagodeservicios
      , div(mpagomiscuentas, cpagomiscuentas) AS m_promedio_pagomiscuentas
      , div(mcajeros_propios_descuentos, ccajeros_propios_descuentos) AS m_promedio_cajeros_propios_descuentos
      , div(mtarjeta_visa_descuentos, ctarjeta_visa_descuentos) AS m_promedio_tarjeta_visa_descuentos
      , div(mtarjeta_master_descuentos, ctarjeta_master_descuentos) AS m_promedio_tarjeta_master_descuentos
      , div(mcomisiones_mantenimiento, ccomisiones_mantenimiento) AS m_promedio_comisiones_mantenimiento
      , div(mcomisiones_otras, ccomisiones_otras) AS m_promedio_comisiones_otras
      , div(mforex_buy, cforex_buy) AS m_promedio_forex_buy
      , div(mforex_sell, cforex_sell) AS m_promedio_forex_sell
      , div(mtransferencias_recibidas, ctransferencias_recibidas) AS m_promedio_transferencias_recibidas
      , div(mtransferencias_emitidas, ctransferencias_emitidas) AS m_promedio_transferencias_emitidas
      , div(mextraccion_autoservicio, cextraccion_autoservicio) AS m_promedio_extraccion_autoservicio
      , div(mcheques_depositados, ccheques_depositados) AS m_promedio_cheques_depositados
      , div(mcheques_emitidos, ccheques_emitidos) AS m_promedio_cheques_emitidos
      , div(mcheques_depositados_rechazados, ccheques_depositados_rechazados) AS m_promedio_cheques_depositados_rechazados
      , div(mcheques_emitidos_rechazados, ccheques_emitidos_rechazados) AS m_promedio_cheques_emitidos_rechazados
      , div(matm, catm_trx) AS m_promedio_atm
      , div(matm_other, catm_trx_other) AS m_promedio_atm_other
      , div(Master_msaldototal, Master_mfinanciacion_limite) AS proporcion_financiacion_master_cubierto
      , div(Master_msaldototal, Master_mlimitecompra) AS proporcion_limite_master_cubierto
      , div(Visa_msaldototal, Visa_mfinanciacion_limite) AS proporcion_financiacion_visa_cubierto
      , div(Visa_msaldototal, Visa_mlimitecompra) AS proporcion_limite_visa_cubierto
      , div(tc_saldototal_total, tc_financiacionlimite_total) AS proporcion_financiacion_total_cubierto
      , div(tc_saldototal_total, tc_limitecompra_total) AS proporcion_limite_total_cubierto
      , div(tc_saldopesos_total, tc_saldototal_total) AS tc_proporcion_saldo_pesos
      , div(tc_saldodolares_total, tc_saldototal_total) AS tc_proporcion_saldo_dolares
      , div(tc_consumopesos_total, tc_consumototal_total) AS tc_proporcion_consumo_pesos
      , div(tc_consumodolares_total, tc_consumototal_total) AS tc_proporcion_consumo_dolares
      , div(tc_consumototal_total, tc_limitecompra_total) AS tc_proporcion_consumo_total_limite_total_cubierto
      , div(tc_pagadopesos_total, tc_pagado_total) AS tc_proporcion_pago_pesos
      , div(tc_pagadodolares_total, tc_pagado_total) AS tc_proporcion_pago_dolares
      , div(tc_adelantopesos_total, tc_adelanto_total) AS tc_proporcion_adelanto_pesos
      , div(tc_adelantodolares_total, tc_adelanto_total) AS tc_proporcion_adelanto_dolares
    FROM sumas
  "
  
  res <- dbGetQuery(con, query)
  as.data.table(res)
}

In [14]:
# Aplico
dataset <- fe_intrames(dataset)

In [15]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

Sys.time()

[1] "2025-12-03 23:27:00 UTC"

### Feature Engineering Historico

In [16]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp



In [17]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [18]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [19]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()

[1] "2025-12-03 23:29:09 UTC"

In [20]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

In [21]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] 1240

[1] "2025-12-03 23:30:24 UTC"

In [22]:
ncol(dataset)
nrow(dataset)
colnames(dataset)

[1] 1240

[1] 4883051

[1] "numero_de_cliente"                                       
   [2] "foto_mes"                                                
   [3] "active_quarter"                                          
   [4] "cliente_vip"                                             
   [5] "internet"                                                
   [6] "cliente_edad"                                            
   [7] "cliente_antiguedad"                                      
   [8] "mrentabilidad"                                           
   [9] "mrentabilidad_annual"                                    
  [10] "mcomisiones"                                             
  [11] "mactivos_margen"                                         
  [12] "mpasivos_margen"                                         
  [13] "cproductos"                                              
  [14] "tcuentas"                                                
  [15] "ccuenta_corriente"                                       
  [16] "mcuenta_corriente_adicional"                             
  [17] "mcuenta_corriente"                                       
  [18] "ccaja_ahorro"                                            
  [19] "mcaja_ahorro"                                            
  [20] "mcaja_ahorro_adicional"                                  
  [21] "mcaja_ahorro_dolares"                                    
  [22] "cdescubierto_preacordado"                                
  [23] "mcuentas_saldo"                                          
  [24] "ctarjeta_debito"                                         
  [25] "ctarjeta_debito_transacciones"                           
  [26] "mautoservicio"                                           
  [27] "ctarjeta_visa"                                           
  [28] "ctarjeta_visa_transacciones"                             
  [29] "mtarjeta_visa_consumo"                                   
  [30] "ctarjeta_master"                                         
  [31] "ctarjeta_master_transacciones"                           
  [32] "mtarjeta_master_consumo"                                 
  [33] "cprestamos_prendarios"                                   
  [34] "mprestamos_prendarios"                                   
  [35] "cprestamos_hipotecarios"                                 
  [36] "mprestamos_hipotecarios"                                 
  [37] "cplazo_fijo"                                             
  [38] "mplazo_fijo_dolares"                                     
  [39] "mplazo_fijo_pesos"                                       
  [40] "cinversion1"                                             
  [41] "minversion1_pesos"                                       
  [42] "minversion1_dolares"                                     
  [43] "cinversion2"                                             
  [44] "minversion2"                                             
  [45] "cseguro_vida"                                            
  [46] "cseguro_auto"                                            
  [47] "cseguro_vivienda"                                        
  [48] "cseguro_accidentes_personales"                           
  [49] "ccaja_seguridad"                                         
  [50] "cpayroll_trx"                                            
  [51] "mpayroll"                                                
  [52] "mpayroll2"                                               
  [53] "cpayroll2_trx"                                           
  [54] "ccuenta_debitos_automaticos"                             
  [55] "mcuenta_debitos_automaticos"                             
  [56] "ctarjeta_visa_debitos_automaticos"                       
  [57] "mttarjeta_visa_debitos_automaticos"                      
  [58] "ctarjeta_master_debitos_automaticos"                     
  [59] "mttarjeta_master_debitos_automaticos"                    
  [60] "cpagodeservicios"                                        
  [61] "mpagodeservicios"                  

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}
* Por experimentos en meses anteriores, se decide cortar en los 11000 registros con mayor probabildiad de POS={"BAJA+1", "BAJA+2"}
* Utilizo *zLightGBM* con 5 canaritos por experiencia de otros experimentos presentados

In [23]:
# training y future
Sys.time()

PARAM$future <- c(202109)

PARAM$train_final$meses <- c(201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201911, 201912,
  202001, 202002, 202003, 202004, 202005,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106, 202107
)
  
PARAM$train_final$undersampling <- 0.05

[1] "2025-12-03 23:30:57 UTC"

### Final Training Strategy

In [25]:
# se filtran los meses donde se entrena el modelo final

library(data.table)
setDT(dataset)

dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$meses]

In [ ]:

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Model

In [28]:
# utilizo  zLightGBM  la nueva libreria
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

Loading required package: zlightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘zlightgbm’”
Installing package into ‘/home/rivas_ae97/.local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: zlightgbm



[1] "2025-12-03 23:32:40 UTC"

In [29]:
# canaritos
PARAM$qcanaritos <- 5

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

for( i in seq(PARAM$qcanaritos) ){
  dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
}

# las columnas canaritos mandatoriamente van al comienzo del dataset
cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )

Sys.time()

[1] "2025-12-03 23:32:45 UTC"

In [30]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c("clase_ternaria", "clase01", "training")
)

In [31]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 250131 columnas 1244 


[1] "2025-12-03 23:32:52 UTC"

In [ ]:
# definicion de parametros

PARAM$lgbm <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= 20L,  

  num_iterations= 9999L, 
  num_leaves= 9999L, 
  learning_rate= 1.0, 
    
  feature_fraction= 0.50,
    
  canaritos= PARAM$qcanaritos, 
  gradient_bound= 0.1  
)

Sys.time()

[1] "2025-12-03 23:32:56 UTC"

####  Entrenamiento del modelo

In [33]:
# entreno el modelo

modelo_final <- lgb.train(
  data= dtrain_final,
  param= PARAM$lgbm
)

Sys.time()

[1] "2025-12-03 23:52:58 UTC"

In [33]:
# grabo el modelo generado, esto pude ser levantado por LighGBM en cualquier maquina
lgb.save(modelo_final, file="zmodelo.txt")

Sys.time()

[1] "2025-11-16 17:56:13 UTC"

### Predict

In [34]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$future]

# penosamente, en la versión actual de zLightGBM  los campos canaritos
#  aunque no se utilizan para nada, también deben estar en el dataset donde se hace el predict()
filas <- nrow(dfuture)

for( i in seq(PARAM$qcanaritos) ){
  dfuture[, paste0("canarito_",i) := runif( filas) ]
}

prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE]),
)

In [ ]:
# tabla de prediccion, puede ser util para futuros ensembles
#  ya que le modelo ganador va a ser un ensemble de LightGBMs

tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "e010_03_prediccion.txt",
  sep= "\t"
)

### Clasificacion

In [ ]:
# Ordenar por probabilidad descendente
setorder(tb_prediccion, -prob)

# Cantidad de envíos
envios <- 11000

# Extraer solo los IDs
ids_envio <- tb_prediccion[1:envios, .(numero_de_cliente)]

# Archivo
archivo_csv <- paste0("e010_03_", envios, ".csv")

# Guardar 
fwrite(
  ids_envio,
  file = archivo_csv,
  col.names = FALSE,  
  sep = ","            
)